# Things that do not cause problem with coordinates
# 1. Opening and saving the structure
# 2. Getting hetero_residues
# 3. Initializing an instance of StructureGraph class
# 4. Creating a graph with .create_graph() method
# 5. Getting cluster of closely located residues with get_clusters() method
# 6. Combining a cluster into single reidues with combine() function
# 7. Filtering according to known criteria (num_atoms, mol_weight)
# 8. Creating structure objects from residues with resi_to_struct function
# 9. Saving structure object with save_ligand_fo_file() function

# so far, was not able to replicate problem

In [8]:
import process
import pocket_search as ps
import nglview
import Bio
from Bio.PDB import PDBParser
import os

In [9]:
structure = process.get_structure('1rqj', process.PDB_PATH, debug=True) # 14 26

Structure 1rqj was loaded from /mnt/c/users/serge/desktop/pdb_download/pdb/rq/pdb1rqj.ent


In [30]:
from process import resi_to_struct


hetero_residues = process.get_hetero_residues(structure)
structure_graph = process.StructureGraph(hetero_residues)
structure_graph.create_graph(distance_threshhold=1.5)
clusters = structure_graph.get_clusters()

residues = []
for cluster in clusters:
    residues.append(process.combine(cluster))

ligands = process.filter_ligands(residues)

ligand_structures = [resi_to_struct(ligand, structure) for ligand in ligands]

process.save_ligand_to_file(ligand_structures, process.OUT_PATH, structure)



In [29]:
view = nglview.show_biopython(residues[-1])
view

NGLWidget()

In [6]:
structure_dir = os.path.join(process.OUT_PATH, structure.id)
cleaned_pdb = ps.clean_pdb(file, structure_dir)
fpocket_dir = ps.fpocket(cleaned_pdb)
barycenters = ps.get_centers(fpocket_dir)
pockets: list = ps.get_pockets(fpocket_dir, num_pockets_threshold=100)
ps.write_bat(cleaned_pdb, fpocket_dir)



***** POCKET HUNTING BEGINS ***** 


mkdir: cannot create directory ‘/mnt/c/users/serge/desktop/2r09/2r09_out/pockets’: File exists


***** POCKET HUNTING ENDS ***** 
/mnt/c/users/serge/desktop/2r09/2r09_out/2r09_info.txt


In [7]:
ligand_structures = [ligand1, ligand2]
filtered_ligands: list = []
for ligand in ligand_structures:
    for pocket in pockets:
        if process.get_distance_to_pocket(ligand, pocket) < 6:
            print(f"d({ligand.id[0]} : {pocket}) = {process.get_distance_to_pocket(ligand, pocket)}")



d(H_4IP : Pocket(number=43, score=-0.127, center=(77.81276658747174, 85.71096824477024, 26.90205528191459))) = 1.0065443224660982
d(H_4IP : Pocket(number=34, score=-0.084, center=(22.02070629678556, 78.02792671573255, -2.33535119833931))) = 1.3021322460175173
